In [1]:
## Slashing Data with Wolverine
# Make sure to read the README associated with this application
# Before Proceeding, some libraries need to be installed
# This application was created to practice tidying data and to learn
# python libraries such as pygame (I wanted to learn GUI)
# If user does not want GUI interface, data code is also seperate


# Import Libraries
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pygame
from pygame.locals import *
from itertools import cycle
from PIL import Image, ImageTk
import sys
from pygame import mixer

## WINDOW CREATION
# Initiate application using Pygame
pygame.init()
clock = pygame.time.Clock()

# Set Window Settings
windowW = 1000
windowH = 667
gamedisplay = pygame.display.set_mode((windowW, windowH))
pygame.display.set_caption("Slashing Data with Wolverine")

# Set Background
Background = pygame.image.load("TidyDataWolverine/MenuBack.png")

# Parameters for Game Box
LIGHT_BEIGE = (245, 245, 220)
box_width, box_height = 750, 450
beige_box = pygame.Surface((box_width, box_height))
beige_box.fill(LIGHT_BEIGE)

box_width, box_height = 750, 450
box_x = 1000 // 2 - box_width // 2
box_y = 667 // 2 - box_height // 2

# Pixel size for border
pixel_size = 10
BROWN = (139, 69, 19)

# Background Sound
mixer.music.load("TidyDataWolverine/MenuMusic.mp3")
pygame.mixer.music.set_volume(0.5)
mixer.music.play(-1)
music_paused = False

## LOAD PNGS
# Create Start & About Buttons
StartButton = pygame.image.load("TidyDataWolverine/StartBase.png")
AboutButton = pygame.image.load("TidyDataWolverine/AboutBase.png")
NextButton = pygame.image.load("TidyDataWolverine/NextButton.png")
MusicOn = pygame.image.load("TidyDataWolverine/sound-on.png")
MusicOff = pygame.image.load("TidyDataWolverine/sound-off.png")
Exit = pygame.image.load("TidyDataWolverine/Exit.png")
ClawButton = pygame.image.load("TidyDataWolverine/claw.png")
ClawButton = pygame.transform.scale_by(ClawButton, 1.5)



#---------
# Data for use
#---------

## DATA MANIPULATION
# Load Data
moneyball = pd.read_csv('mutant_moneyball.csv')

# Create dataframe
dataframe = pd.DataFrame(moneyball)

# Melt Data (Wide Format to Long Format)
dataframemelted = dataframe.melt(id_vars =['Member'], var_name = 'Company', value_name = 'Total Value ($)')


# Use extract to split first and last name into two rows
dataframemelted[['First','Last']] = dataframemelted['Member'].str.extract(r'([a-z]+)([A-Z][a-z]+)')

# Remove old member column
dataframemelted.drop(columns=['Member'], inplace=True)


# Utilize Split Twice, To split decade and Company name
dataframemelted[['Decade', 'Temperory']] = dataframemelted['Company'].str.split('_', expand=True)


dataframemelted.drop(columns=['Temperory'], inplace=True)

#------------
# Temporary Visuals will be created throughout for scenes
#------------

## CREATED USING .melt, .drop, and .split

# First Tempory Visuals
dataframemeltedincomplete = dataframemelted

# Remove Unnecessary name classifications from variables
dataframemelted['Company'] = dataframemelted['Company'].str.replace('TotalValue', '', regex=False)

dataframemelted['Company'] = dataframemelted['Company'].str[4:]

dataframemelted['Decade'] = dataframemelted['Decade'].str.replace('TotalValue', '', regex=False)

# Had to ensure all the values were strings to delete $
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].astype(str)

# Delete unnecessary $
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].str.replace('$', '', regex=False)

# lower case variables for easier manipulation/search
dataframemelted['Last'] = dataframemelted['Last'].str.lower()

dataframemelted['Company'] = dataframemelted['Company'].str.lower()


# Reorder for logic
dataframemelted = dataframemelted[['First', 'Last', 'Company', 'Decade', 'Total Value ($)']]

## CREATED USING str.replace, .astype, and str.lower

# Second Temporary Visuals
dataframemeltedincomplete2 = dataframemelted


# Empty in Total Value variables will be replaced with the mean of the company's sales
dataframemelted['Total Value ($)'] = pd.to_numeric(dataframemelted['Total Value ($)'], errors='coerce')

# Calculate the total earnings per character within each company across all decades
character_company_total = dataframemelted.groupby(['First', 'Last', 'Company'])['Total Value ($)'].transform('sum')

# Compute the per-decade average (divide total earnings by 4 decades)
character_company_avg = character_company_total / 4

# Replace NaN values with the per-character per-company average
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].fillna(character_company_avg)

dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].round(2)

# Delete missing memembers
dataframemelted = dataframemelted.dropna(subset=['First'])

# Final dataframemelted is the final visualization

## CREATED USING .to_numeric, .grouby, .fillna, and .round



#-----------
# THIS SECTION PRIORITIZES VISUALIZATION
#-----------

## FUNCTIONS & CLASSES
# Create Border Function
def draw_pixelated_border(surface):
    offset = pixel_size  # Border thickness

    # Top and bottom borders
    for x in range(box_x - offset, box_x + box_width + offset, pixel_size):
        pygame.draw.rect(surface, BROWN, (x, box_y - offset, pixel_size, pixel_size))  # Top
        pygame.draw.rect(surface, BROWN, (x, box_y + box_height, pixel_size, pixel_size))  # Bottom

    # Left and right borders
    for y in range(box_y - offset, box_y + box_height + offset, pixel_size):
        pygame.draw.rect(surface, BROWN, (box_x - offset, y, pixel_size, pixel_size))  # Left
        pygame.draw.rect(surface, BROWN, (box_x + box_width, y, pixel_size, pixel_size))  # Right


# Create Button Class
class Button():
    def __init__(self, x, y, image):
        self.image = image
        self.rect = self.image.get_rect()
        self.rect.topleft = (x,y)
        self.clicked = False

    def draw(self):
        action = False
        pos = pygame.mouse.get_pos()

        # Check Click Condition
        if self.rect.collidepoint(pos):
            if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
                self.clicked = True
                action = True
            elif pygame.mouse.get_pressed()[0] == 0:
                self.clicked = False

        # Draw Button on Screen
        gamedisplay.blit(self.image, (self.rect.x, self.rect.y))

        return action
    
# Create Text Render Function
lastcharupdate = pygame.time.get_ticks()
def displaytext():
    global currenttext, charnum, lastcharupdate, textcomplete

    now = pygame.time.get_ticks()
    if not textcomplete and now - lastcharupdate > textspeed:
        if charnum < len(dialogue[dialogueNum]):
            currenttext += dialogue[dialogueNum][charnum]
            charnum += 1
            lastcharupdate = now
        else:
            textcomplete = True
            

# Create Text Display Function For Data Section
def displayminitext(text, x, y):
    font = pygame.font.Font(None, 32)
    text_surface = font.render(text, True, (0, 0, 0))
    gamedisplay.blit(text_surface, (x, y))

# Create Display Data Function
def displaydata(df, x_offset):
    y = 50
    for index, row in dataframe.iterrows():
        rowtext = " | ".join(str(value) for value in row)
        displayminitext(rowtext, x_offset, y)
        y += 25

# This function provided a column label and first row for active display
def displayheaders(df, y_offset):
    font = pygame.font.Font(None, 36)
    x_offset = 130
    border_color = (0, 0, 0) 
    text_color = (0, 0, 0)  

    # Display headers with borders
    headers = df.columns.tolist()
    for header in headers:
        header_text = str(header)
        text_surface = font.render(header_text, True, text_color)
        gamedisplay.blit(text_surface, (x_offset, y_offset))
        
        # Draw border around the header
        header_rect = pygame.Rect(x_offset - 5, y_offset - 5, 150, 40)  
        pygame.draw.rect(gamedisplay, border_color, header_rect, 2)  
        x_offset += 150  # Space between columns
    
    y_offset += 40  

   
    first_row = df.iloc[0] 
    x_offset = 130  
    for value in first_row:
        row_text = str(value)
        text_surface = font.render(row_text, True, text_color)
        gamedisplay.blit(text_surface, (x_offset, y_offset))
        
        # Draw border around the data value
        row_rect = pygame.Rect(x_offset - 5, y_offset - 5, 150, 40)  
        pygame.draw.rect(gamedisplay, border_color, row_rect, 2)  
        x_offset += 150

# Create FinalDataFrame Display Class:
class DataFrameDisplay:
    def __init__(self, dataframemelted):
        self.dataframemelted = dataframemelted
        self.scroll = 0
        self.row_h = 30

    def draw(self, gamedisplay):
        # Background for data table
        pygame.draw.rect(gamedisplay, (255, 255, 255), (125, 108, 750, 450))  # Background Color (White)
        columns = list(self.dataframemelted.columns)
        y = 108

        # Draw rows of the DataFrame
        for i, row in enumerate([columns] + self.dataframemelted.iloc[self.scroll:self.scroll + 450 // self.row_h].values.tolist()):
            pygame.draw.rect(gamedisplay, (200, 200, 200) if i == 0 else (225, 225, 225), (125, y, 750, self.row_h))
            for j, val in enumerate(row):
                gamedisplay.blit(pygame.font.SysFont("Arial", 12).render(str(val), True, (0, 0, 0)),
                                 (125 + j * (750 // len(columns)) + 5, y + 5))
            y += self.row_h

    def scroll_dataframemelted(self, d):
        self.scroll = max(0, min(self.scroll + d, len(self.dataframemelted) - 450 // self.row_h))


## IMPORT SPRITES - CREATE PARAMETERS
# Sprite Sheet Initialization
wolverine_idle = pygame.image.load("TidyDataWolverine/WolverineSprites.png").convert_alpha()
wolverine_stand = pygame.image.load("TidyDataWolverine/WolverineIdle.png")
menu_spritesheet = pygame.image.load("TidyDataWolverine/MenuTextSpriteSheet.png")
SpeechBubble = pygame.image.load("TidyDataWolverine/Speech5.gif")

# Menu Paramters

MenuWidth = 800
MenuHeight = 354

# Wolverine Parameters

WolverineWidth = 127
WolverineHeight = 128
Columns = 5

# Speech Bubble Parameters

SpeechFiveWidth = 639
SpeechFiveHeight = 180
fontsize = 28
textspeed = 25

# Dialogue Control
dialogueNum = 0
charnum = 0
textcomplete = False

# Create Long Part of Code - Start Scene
textOne = "Hey there, Bub. If you haven't figure it out, I'm Wolverine."
textTwo = "The other X-Men's data have been corruped in this Universe."
textThree = "While I can slash through this messy data by myself..."
textFour = "I need you to tell me how to do it."
textFive = "I need your help to tidy up this mess."
textSix = "Let's get started, Bub."
dialogue = [textOne, textTwo, textThree, textFour, textFive, textSix]

currenttext = ""

textSeven = "The data is too wide to slash, press space to melt it!"





font = pygame.font.Font(None, fontsize)
text_index = 0


# Idle Configuration
IdleStart = 3
IdleEnd = 4
AnimateSpeed = 85
MenuSpeed = 120

# Slashing Configuration
SlashStart = 5
SlashEnd = 7


# Frame Extraction for Menu Text Sprite
MenuFrames = []
for i in range(Columns):
    x = i * MenuWidth
    MenuFrame = menu_spritesheet.subsurface(pygame.Rect(x,0, MenuWidth, MenuHeight))
    MenuFrames.append(MenuFrame)


# Frame Extraction For Wolverine Idle Sprite
Wolverineframes = []
for row in range(IdleStart, IdleEnd + 1):
    for col in range(Columns):
        x = col*WolverineWidth
        y = row*WolverineHeight
        Wolverineframe = wolverine_idle.subsurface(pygame.Rect(x,y, WolverineWidth, WolverineHeight))
        Wolverineframes.append(Wolverineframe)

# Frame Extraction For Wolverine Slashing Sprite
WolverineSlashingframes = []
for row in range(SlashStart, SlashEnd + 1):
    for col in range(Columns):
        x = col*WolverineWidth
        y = row*WolverineHeight
        WolverineSlashingframe = wolverine_idle.subsurface(pygame.Rect(x,y, WolverineWidth, WolverineHeight))
        WolverineSlashingframes.append(WolverineSlashingframe)

# Animation Definitions
WolverineIndex, MenuIndex, SpeechIndex = 0, 0, 0
updatelast = pygame.time.get_ticks()
lastUpdateWolverine = pygame.time.get_ticks()
lastUpdateMenu = pygame.time.get_ticks()
lastUpdateSpeech = pygame.time.get_ticks()

# Scene Variables
Menu = True
About = False
Start = False
Data = False
Melt = False
Done = False
MiniDone = False
CharacterClear = False
ClearCharacters = False
WolverineMusic = False
FinalEvent = False
SpeechNumber = 0

#-------
# GAME LOOP :D
#-------

# Game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    currenttime = pygame.time.get_ticks()
    
    # Update Idle Wolverine
    if currenttime - lastUpdateWolverine > AnimateSpeed:
        WolverineIndex = (WolverineIndex + 1) % len(Wolverineframes)
        lastUpdateWolverine = currenttime

    # Update Menu Animation
    if currenttime - lastUpdateMenu > MenuSpeed:
        MenuIndex = (MenuIndex + 1) % len(MenuFrames) 
        lastUpdateMenu = currenttime

            
    
    if Menu:
        # Button instances
        # About button not used
        start_button = Button(600, 500, StartButton)
        music_button = Button(850, 15, MusicOn)


        gamedisplay.blit(Background, (0,0))
        gamedisplay.blit(Wolverineframes[WolverineIndex], (350, 395))
        gamedisplay.blit(MenuFrames[MenuIndex], (0,0))

        if music_button.draw():
            if music_paused:
                mixer.music.unpause()
                music_button.image = MusicOn
            else:
                mixer.music.pause()
                music_button.image = MusicOff
            music_paused = not music_paused

        
    if start_button.draw():
        Menu = False
     
        Start = True

    elif About:
        # Display brief about page
        gamedisplay.blit(Background, (0,0))
        exit_button = Button(350, 400, Exit)
        start_button = Button(1200, 500, StartButton)
        about_button = Button(1200, 510, AboutButton)
        if exit_button.draw():
            Menu = True
            About = False
    elif Start:
        # Create the start of experience
        gamedisplay.blit(Background, (0,0))
        start_button = Button(1200, 500, StartButton)
        about_button = Button(1200, 510, AboutButton)
        next_button = Button(650, 450, NextButton)

        currenttime = pygame.time.get_ticks()
        
        # Control Music
        if music_button.draw():
            if music_paused:
                mixer.music.unpause()
                music_button.image = MusicOn
            else:
                mixer.music.pause()
                music_button.image = MusicOff
            music_paused = not music_paused
        
        scaledwolverine_stand = pygame.transform.scale_by(wolverine_stand, 2.5)
        if not WolverineMusic:
            pygame.mixer.music.load("TidyDataWolverine/WolverineMusic.mp3")
            pygame.mixer.music.set_volume(1)
            mixer.music.play(-1)
            WolverineMusic = True
        
        gamedisplay.blit(beige_box, (1000//2 - box_width//2, 667//2 - box_height//2))
        draw_pixelated_border(gamedisplay)

        gamedisplay.blit(SpeechBubble, (175, 150))
        
        gamedisplay.blit(scaledwolverine_stand, (125, 250))

        displaytext()
        textsurface = font.render(currenttext, True, (0,0,0))
        gamedisplay.blit(textsurface, (205, 200))

    

        if next_button.draw():
            if textcomplete:
                dialogueNum += 1
                if dialogueNum < len(dialogue):
                    currenttext = ""
                    charnum = 0
                    textcomplete = False
                else:
                    Start = False
                    Data = True

            
    elif Data:
        gamedisplay.blit(Background, (0,0))
        start_button = Button(1200, 500, StartButton)
        about_button = Button(1200, 510, AboutButton)

        currenttime = pygame.time.get_ticks()

        if music_button.draw():
            if music_paused:
                mixer.music.unpause()
                music_button.image = MusicOn
            else:
                mixer.music.pause()
                music_button.image = MusicOff
            music_paused = not music_paused
        
        scaledwolverine_stand = pygame.transform.scale_by(wolverine_stand, 2.5)
        if not WolverineMusic:
            mixer.music.load("TidyDataWolverine/WolverineMusic.mp3")
            mixer.music.play(-1)
            WolverineMusic = True

        gamedisplay.blit(beige_box, (1000//2 - box_width//2, 667//2 - box_height//2))
        draw_pixelated_border(gamedisplay)
        gamedisplay.blit(Wolverineframes[WolverineIndex], (150, 395))

        gamedisplay.blit(SpeechBubble, (175, 110))


        # Start Events Dependent on User
        if MiniDone:
            df_display = DataFrameDisplay(dataframemelted)
            df_display.draw(gamedisplay)
            for e in pygame.event.get():
                if e.type == pygame.QUIT:
                    running = False
        elif Done:
            next_button = Button(650,450, NextButton)
            displayminitext("You did it! You tidied up the X-Men's data, Bub", 215, 145)
            displayminitext("There used to be missing data, jumbled variables", 215, 165)
            displayminitext(", and mixed variable types.", 215, 185)
            displayheaders(dataframemelted, 300)
            
            if next_button.draw():
                MiniDone = True
                
        elif FinalEvent:
            displayminitext("Just do some .to_numeric, .grouby, .fillna, and .round!", 215, 145)
            displayminitext("We almost have this mess cleaned up! One more slash!", 215, 165)
            displayheaders(dataframemeltedincomplete2, 300)
            
            
            claw_button = Button(700, 450, ClawButton)

            if claw_button.draw():
                Done = True
                claw_button = Button(2000, 450, ClawButton)
                

        elif CharacterClear:
            next_button = Button(2000,500, NextButton)
            
            claw_button = Button(700, 450, ClawButton)
            
            if ClearCharacters: 
                displayminitext("That's better!", 215, 145)
                displayminitext("Thats what str.replace, .astype, and str.lower can do!", 215, 165)
                displayheaders(dataframemeltedincomplete2, 300)
                claw_button = Button(2000, 450, ClawButton)
                next_button = Button(650,450, NextButton)
                
                
                
                if next_button.draw():
                    next_button = Button(2000,500, NextButton)
                    pygame.time.delay(5000)
                    FinalEvent  = True

                    
            else: 
                displayminitext("There is too much extra data in the rows!", 215, 145)
                displayminitext("Press the claw button to slash that data!", 215, 165)
                displayheaders(dataframemeltedincomplete, 300)
                
    
            if claw_button.draw():
                claw_button = Button(2000, 450, ClawButton)
                ClearCharacters = True
            
        
        elif Melt:
            displayminitext("You melted the data with .melt and", 215, 145)
            displayminitext("restructed with .drop and .split", 215, 165)
            displayheaders(dataframemeltedincomplete, 300)
            next_button = Button(650, 450, NextButton)
            if next_button.draw():
                CharacterClear = True
                Melt = False
                pygame.time.delay(500)
                

        elif not Melt:
            displayminitext(textSeven, 215, 155)
            displayheaders(dataframe, 300)

        for event in pygame.event.get():
            if event.type ==pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    Melt = not Melt


        pygame.display.update()
        
       
        
    
       
        
    pygame.display.update()
    clock.tick(30)

pygame.quit()
sys.exit()


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.32.50, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\gingw\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
